In [1]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer, CrossEncoder
from langchain.prompts.prompt import PromptTemplate
from langchain_ollama import ChatOllama

C:\Users\91629\AppData\Roaming\Python\Python312\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
embedding_model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
reranking_model = CrossEncoder("mixedbread-ai/mxbai-rerank-xsmall-v1")

pc = Pinecone(api_key="abcdefghijklmnojjnfvjknsdvsv")

index_name = "ramayana-embeddings"
index = pc.Index(index_name)

## Enter your Query here

#### In total, takes 5 minutes to answer a query

In [3]:
query = "How was Ram's personality, good or bad?"

In [4]:
xq = embedding_model.encode(query).tolist()
top_results = index.query(vector=xq, top_k=10, include_metadata=True)
top_scores = top_results.matches  # Get the top 10 matches
documents = [match['metadata']['text'] for match in top_scores]

In [5]:
final_results = reranking_model.rank(query, documents, return_documents=True, top_k=5)

In [6]:
information = final_results[0]['text']

In [7]:
summary_template = """
    Given the information {information} about a prompt, I want to answer the given query from the Indian epic Ramayana: {query}.
    You can use your knowledge along with the provide information to answer the question. However, it it given that the information passed here
    is exactly correct, although it may have some spelling mistakes. Despite all these, if you don't know the answer, simply say No. Don't try to generate
    wrong answers to the given query.
"""

In [8]:
summary_prompt_template = PromptTemplate(
    input_variables=["information"], 
    template=summary_template
)

In [9]:
llm = ChatOllama(model='llama3', temperature=0.3)

In [10]:
#   Will take 4 to 5 minutes to answer. Also, your laptop may lag a bit during running this...

chain = summary_prompt_template | llm
res = chain.invoke(input= {"information": information,
                           "query": query})

In [11]:
print(res.content)

Based on the provided information from the Indian epic Ramayana, I can conclude that Rama's personality is overwhelmingly good. The description highlights his virtues, such as:

* Being virtuous and true to his promises
* Being benevolent to his subjects and omniscient
* Being pure and responsive to devotion
* Being equal to Brahma in protecting his people and pleasing to look upon
* Having infallible memory and being beloved by all
* Possessing excellent attributes, such as patience, generosity, and truthfulness

The text also describes Rama's actions, such as going into exile to fulfill his father's promise and obey the command of his royal sire. This shows that he is a dutiful son and a responsible individual.

There is no mention or indication of any negative qualities or behaviors in Rama's personality. Therefore, I can confidently say that Rama's personality is good.


In [16]:
res

AIMessage(content="Based on the provided information from the Indian epic Ramayana, I can conclude that Rama's personality is overwhelmingly good. The description highlights his virtues, such as:\n\n* Being virtuous and true to his promises\n* Being benevolent to his subjects and omniscient\n* Being pure and responsive to devotion\n* Being equal to Brahma in protecting his people and pleasing to look upon\n* Having infallible memory and being beloved by all\n* Possessing excellent attributes, such as patience, generosity, and truthfulness\n\nThe text also describes Rama's actions, such as going into exile to fulfill his father's promise and obey the command of his royal sire. This shows that he is a dutiful son and a responsible individual.\n\nThere is no mention or indication of any negative qualities or behaviors in Rama's personality. Therefore, I can confidently say that Rama's personality is good.", response_metadata={'model': 'llama3', 'created_at': '2024-07-29T04:53:26.5178134Z'

In [20]:
res.usage_metadata

777

In [21]:
new_result = {
    "query": query,
    "result": res.content,
    "source_documents": final_results[0]['text']
}

In [22]:
new_result

{'query': "How was Ram's personality, good or bad?",
 'result': "Based on the provided information from the Indian epic Ramayana, I can conclude that Rama's personality is overwhelmingly good. The description highlights his virtues, such as:\n\n* Being virtuous and true to his promises\n* Being benevolent to his subjects and omniscient\n* Being pure and responsive to devotion\n* Being equal to Brahma in protecting his people and pleasing to look upon\n* Having infallible memory and being beloved by all\n* Possessing excellent attributes, such as patience, generosity, and truthfulness\n\nThe text also describes Rama's actions, such as going into exile to fulfill his father's promise and obey the command of his royal sire. This shows that he is a dutiful son and a responsible individual.\n\nThere is no mention or indication of any negative qualities or behaviors in Rama's personality. Therefore, I can confidently say that Rama's personality is good.",
 'source_documents': 'proportioned l